In [ ]:
import os
import cv2
import math

# --- Directories ---
input_video_dir = "../../videos/Indoor/"
mot_dir = "../../BoT-SORT_outputs/Indoor/filtered/merged/"
att_dir = "../../BoT-SORT_outputs/Indoor/transformed/merged_with_attributes/"
color_dir = "../../ground_truth/Indoor/transformed_MOT"
output_dir = "../../videos/Indoor/minimap_drawn"
os.makedirs(output_dir, exist_ok=True)

# Video directory for minimap
pred_dir = "../../videos/Indoor/minimap/prediction"
gt_dir   = "../../videos/Indoor/minimap/ground_truth"

# Color conversion mapping for MOT（lowercase）
mot_color_map = {
    "red": (0, 0, 204),
    "blue": (204, 0, 0),
    "pink": (144, 84, 204),
    "black": (0, 0, 0),
    "orange": (0, 120, 204),
    "yellow": (108, 204, 176),  # yellowはgreenとして解釈
}

# --- File read function ---
def load_mot_file(path):
    mot_dict = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 6:
                continue
            try:
                frame = int(float(parts[0]))  
                track_id = int(float(parts[1]))
                x = float(parts[2])
                y = float(parts[3])
                w = float(parts[4])
                h = float(parts[5])
            except Exception as e:
                print(f"parse error: {line}")
                continue
            if frame not in mot_dict:
                mot_dict[frame] = {}
            mot_dict[frame][track_id] = (x, y, w, h)
    return mot_dict

def load_att_file(path):
    att_dict = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 5:
                continue
            try:
                frame = int(float(parts[0]))  
                track_id = int(float(parts[1]))
                x = float(parts[2])
                y = float(parts[3])
                attribute = parts[-1]
            except Exception as e:
                print(f"Attribute file parsing error: {line}")
                continue
            if frame not in att_dict:
                att_dict[frame] = {}
            att_dict[frame][track_id] = (x, y, attribute)
    return att_dict

def load_color_file(path):
    color_dict = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 5:
                continue
            try:
                frame = int(float(parts[0]))  
                track_id = int(float(parts[1]))
                x = float(parts[2])
                y = float(parts[3])
                color_str = parts[4].lower()
            except Exception as e:
                print(f"Color file parsing error: {line}")
                continue
            if frame not in color_dict:
                color_dict[frame] = {}
            color_dict[frame][track_id] = (x, y, color_str)
    return color_dict

def build_track_color_mapping(att_data, color_data):
    mapping = {}
    att_frame = 1
    color_frame = 1
    if att_frame not in att_data or color_frame not in color_data:
        print("Data for the first frame is missing.")
        return mapping
    att_tracks = att_data[att_frame]
    color_tracks = color_data[color_frame]
    for att_tid, (ax, ay, _) in att_tracks.items():
        min_dist = float("inf")
        matched_color = None
        for color_tid, (cx, cy, color_str) in color_tracks.items():
            dist = math.hypot(ax - cx, ay - cy)
            if dist < min_dist:
                min_dist = dist
                matched_color = color_str
        if matched_color is not None:
            mapping[att_tid] = mot_color_map.get(matched_color, (0, 0, 0))
    return mapping

# --- Each video processing loop ---
for video_file in os.listdir(input_video_dir):
    if not video_file.endswith(".mp4"):
        continue

    video_path = os.path.join(input_video_dir, video_file)
    mot_path = os.path.join(mot_dir, os.path.splitext(video_file)[0] + ".txt")
    att_path = os.path.join(att_dir, os.path.splitext(video_file)[0] + ".txt")
    color_path = os.path.join(color_dir, os.path.splitext(video_file)[0] + "_color.txt")
    
    pred_video_path = os.path.join(pred_dir, video_file)
    gt_video_path   = os.path.join(gt_dir, video_file)
    
    if not os.path.exists(mot_path):
        print(f"MOT file not found: {mot_path}")
        continue
    if not os.path.exists(att_path):
        print(f"Attribute file not found: {att_path}")
        continue
    if not os.path.exists(color_path):
        print(f"Color file not found: {color_path}")
        continue
    if not os.path.exists(pred_video_path):
        print(f"Prediction video not found: {pred_video_path}")
        continue
    if not os.path.exists(gt_video_path):
        print(f"Ground Truth video not found: {gt_video_path}")
        continue

    mot_data = load_mot_file(mot_path)      
    att_data = load_att_file(att_path)        
    color_data = load_color_file(color_path)  
    track_color_mapping = build_track_color_mapping(att_data, color_data)
    
    cap = cv2.VideoCapture(video_path)
    pred_cap = cv2.VideoCapture(pred_video_path)
    gt_cap = cv2.VideoCapture(gt_video_path)
    
    if not cap.isOpened():
        print(f"Could not open video: {video_path}")
        continue

    main_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    main_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_video_path = os.path.join(output_dir, video_file)
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (main_width, main_height))
    
    # Set M to 1/5 of the width of the main video
    M = main_width / 5.0
    L = (4/3.0) * M
    
    # Resize the mini-map video to width M, keeping the original aspect ratio
    # First, get the original size from one frame of the Prediction video
    ret_pred, first_pred = pred_cap.read()
    if not ret_pred:
        print(f"Failure to acquire frames for Prediction video: {pred_video_path}")
        continue
    orig_pred_h, orig_pred_w = first_pred.shape[:2]
    # Aspect ratio r = h / w, thus height after resizing H = M * r
    r = orig_pred_h / orig_pred_w
    H = M * r
    # reset
    pred_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    gt_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    comp_width = (7/3.0) * M
    pred_x = int(L)
    gt_x = int(L + M + (M/3.0))

    label_space = 30
    overlay_y = main_height - int(H + label_space)
    
    # --- Font settings for labels ---
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    label_color = (0, 0, 0)  # black

    frame_index = 0
    while True:
        ret_main, main_frame = cap.read()
        if not ret_main:
            break

        current_mot_frame = frame_index         
        current_att_frame = frame_index + 1       
        if current_mot_frame in mot_data:
            for track_id, bbox in mot_data[current_mot_frame].items():
                x, y, w, h = bbox
                a = w / 2
                b = a / 5
                cx = int(x + w/2)
                cy = int(y + h + b/2)
                color = track_color_mapping.get(track_id, (0, 0, 0))
                cv2.ellipse(main_frame, (cx, cy), (int(a), int(b)), 0, 330, 570, color, 2)
                attribute = ""
                if current_att_frame in att_data and track_id in att_data[current_att_frame]:
                    attribute = att_data[current_att_frame][track_id][2]
                text_pos = (int(x), int(y - 10))
                cv2.putText(main_frame, attribute, text_pos,
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
        
        # --- mini-map overlay ---
        ret_pred, frame_pred = pred_cap.read()
        ret_gt, frame_gt = gt_cap.read()
        if ret_pred and ret_gt:
            # mini-map resizes to width M while maintaining aspect ratio
            frame_pred_resized = cv2.resize(frame_pred, (int(M), int(H)), interpolation=cv2.INTER_LINEAR)
            frame_gt_resized   = cv2.resize(frame_gt,   (int(M), int(H)), interpolation=cv2.INTER_LINEAR)
            alpha = 0.8  # Semi-transparent composite rate
            
            # Predictionを main_frame 上の位置へ配置：位置は x=pred_x, y=overlay_y
            roi_pred = main_frame[overlay_y:overlay_y+int(H), pred_x:pred_x+int(M)]
            # The sizes should match, so they are combined as they are
            blended_pred = cv2.addWeighted(roi_pred, 1 - alpha, frame_pred_resized, alpha, 0)
            main_frame[overlay_y:overlay_y+int(H), pred_x:pred_x+int(M)] = blended_pred
            
            # Ground Truth Placement
            roi_gt = main_frame[overlay_y:overlay_y+int(H), gt_x:gt_x+int(M)]
            blended_gt = cv2.addWeighted(roi_gt, 1 - alpha, frame_gt_resized, alpha, 0)
            main_frame[overlay_y:overlay_y+int(H), gt_x:gt_x+int(M)] = blended_gt
            
            # --- Label Drawing ---
            # Labels are placed in black letters at the bottom of the mini-map. The x coordinate of the label is in the center of each mini-map.
            pred_label_x = pred_x + int(M/2)
            gt_label_x = gt_x + int(M/2)
            label_y = overlay_y + int(H) + 25  
            cv2.putText(main_frame, "Prediction", (pred_label_x - 75, label_y), font, font_scale, label_color, font_thickness)
            cv2.putText(main_frame, "Ground Truth", (gt_label_x - 100, label_y), font, font_scale, label_color, font_thickness)
            
        out.write(main_frame)
        frame_index += 1

    cap.release()
    pred_cap.release()
    gt_cap.release()
    out.release()
    print(f"{video_file} processing is complete. Output: {output_video_path}")


basket_S6T3_post.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S6T3_post.mp4
basket_S6T4_post.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S6T4_post.mp4
basket_S6T5_post.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S6T5_post.mp4
basket_S6T6_post.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S6T6_post.mp4
basket_S6T7_post.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S6T7_post.mp4
basket_S1T1_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T1_pre.mp4
basket_S1T2_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T2_pre.mp4
basket_S1T3_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T3_pre.mp4
basket_S1T4_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T4_pre.mp4
basket_S1T5_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T5_pre.mp4
basket_S1T6_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T6_pre.mp4
basket_S1T7_pre.mp4 の処理完了。出力: ../videos/Indoor/minimap_drawn/basket_S1T7_pre.mp4
basket_S2T1_pre.mp